### Can we classify each phase as the stress vs non-stress phase for each subject? ###

Note: Incomplete

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import pandas as pd
import random
import scipy.signal as ss
import sys

import tools.data_reader_sfi as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

stress_phases = [f"CLIP-{str(phase)}" for phase in dr.CLIP_INDICES]
rest_phase = ["BIOFEEDBACK-REST"]


c:\Users\zhoux\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
importlib.reload(train)
importlib.reload(dt)
importlib.reload(preprocessing)


metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
    train.Metrics.RESP,
    # train.Metrics.MEAN_WRIST_ACT, 
    # train.Metrics.PEAK_WRIST_ACC 
]

model_phases = stress_phases + rest_phase

normalize = True
# normalize = False
x, y = train.Train_SFI.get_sfi_data(metrics, model_phases, verbose=False, normalize=normalize)
print(x.shape)
print(y.shape)
# 0-1 scaling
for i in range(2, len(x.columns)):
    if x.columns[i] in metrics:
        data_col = x[x.columns[i]]
        data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
        x[x.columns[i]] = data_col

models = {
    "SVM": SVC(C=10, gamma=0.01), 
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "LogReg": LogisticRegression(max_iter=1000),
    "Bayes": GaussianNB(),
    "DT": DecisionTreeClassifier()
}

results = {
    "SVM": [], 
    "KNN": [],
    "LogReg": [],
    "Bayes": [],
    "DT": []
}
num_iters = 10
for _ in range(num_iters):
    out = train.train_predict(models, x, y, by_subject=False, test_size=0.12)
    for model_name in results:
        results[model_name].append(out[model_name])

for model_name in results.keys():
    print(f"{model_name} accuracy over {num_iters} rounds: {np.mean(results[model_name])}")

(56, 10)
(56, 2)
(50, 10)
(50, 2)
(6, 10)
(6, 2)


ValueError: Input y contains NaN.